In [ ]:
import numpy as np
import pandas as pd 

#Import libraries

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

#Import Databases

In [ ]:
movies = movies.merge(credits,on='title')

#Merge both databases on the basis of "title"

In [ ]:
movies.shape #Check our data's dimensions

In [ ]:
movies.head(1)

#Check how merged databses look like

In [ ]:
#genres
#id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

#Select relevant fields

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

#Remove blanks

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres 

#View how genres appear for a movie

In [ ]:
movies.head()

In [ ]:
import ast
ast.literal_eval

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

#Get all values from the dictionary

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1;
        else:
            break
    return L

#Get top 3 values from the dictionary

In [ ]:
def fetch_director(obj):
    L = []
    
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

#Fetch Diretor's name from the dictonary

In [ ]:
movies['genres']= movies['genres'].apply(convert)

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)


In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)


In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())  #Lamda split converts string to a list of separate words

In [ ]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x]) 

#Deletion of spaces

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] 

#Create new field "tags" which has all textual data that we need to measure similarity

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']] #New database with combined field "tags"

In [ ]:
new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x)) #convert all separate strings into a single string

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower()) #lowercase conversion

In [ ]:
new_df.head()

#How our database looks presently

In [ ]:
import nltk

In [ ]:
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y= []
    for i in text.split():
       y.append(ps.stem(i));
    return " ".join(y);

#Split string into list of words, stem each word to its root, and join it back to a complete single string

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem) #Call the function

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english') # Take 5000 most popular words from all of them. Remove stop words to add precision

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray() 

#Tokenize and count the occurrence of each word in the 'tags' column, creating a matrix of token counts. Convert the matrix to a dense NumPy array.

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity= cosine_similarity(vectors) #Measure dot product between rows of matrix to check for similarity between vectors

In [ ]:
similarity[0] #Check for similarity

In [ ]:
new_df[new_df['title'] == 'Batman Begins'].index[0]

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6] 

#Sort list in descending order of similarity, present top 5
#lambda function for changing sorting key from list no. to similarities

In [ ]:
def give_recommendations_like(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6] 

    for i in movies_list:
        print(new_df.iloc[i[0]].title) #i[0] gives Sno of movies, iloc allows to access name

In [ ]:
give_recommendations_like('Interstellar') # You're all set!Enter a movie name in the quotes. 